# Opção B – LSTM Compartilhada por Sensor
Notebook de demonstração mostrando como o código em `codes/` atende aos requisitos descritos no **description.pdf** (Fundamentos de Redes Neurais – Tarefa Extra).

## 1 — Ambiente e Importações
Reutilizamos os módulos utilitários existentes e a classe `LSTMModel` definida em `codes/LSTM.py`.

In [1]:
import random, os, json, pathlib, math, matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import losses

from codes.data_utils import load_csv_parts, window_generator, train_val_test_split
from codes.LSTM import LSTMModel, F1Score
from codes.ModelEvaluator import ModelEvaluator

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

2025-07-29 20:46:49.033387: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-29 20:46:49.043104: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-29 20:46:49.117117: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-29 20:46:49.164759: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753832809.214040    4074 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753832809.22

ModuleNotFoundError: No module named 'codes'

## 2 — Hiperparâmetros (do PDF)

In [ ]:
WINDOW = 240  # recomendado no PDF
STEP = 1
BATCH = 128
EPOCHS = 100
LR = 1e-3
DATASET_DIR = '../dataset'  # ajuste conforme necessário
EXP_DIR = pathlib.Path('notebook_runs/exp_pt')
EXP_DIR.mkdir(parents=True, exist_ok=True)

## 3 — Carregamento dos dados e geração das janelas

In [ ]:
print('Carregando partes CSV …')
df = load_csv_parts(DATASET_DIR)
print(f'Linhas carregadas: {len(df):,}.')

print('Gerando janelas …')
X, y = window_generator(df, window_size=WINDOW, step_size=STEP)
print(f'Janelas: {X.shape}, positivos: {y.sum()} ({y.mean():.4%})')

X_train, y_train, X_val, y_val, X_test, y_test = train_val_test_split(X, y)

# Expande dimensões para o LSTM
X_train_e = np.expand_dims(X_train, -1)
X_val_e = np.expand_dims(X_val, -1)
X_test_e = np.expand_dims(X_test, -1)

## 4 — Pesos de Classe e Função de Perda

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weight_vals = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights = {0: class_weight_vals[0], 1: class_weight_vals[1]}
print('Pesos de classe:', class_weights)
loss_fn = losses.BinaryFocalCrossentropy()  # ou BCE

## 5 — Construção do Modelo

In [ ]:
model_wrapper = LSTMModel(
    window_size=WINDOW,
    metrics=[Precision(name='precision'), Recall(name='recall'), F1Score()],
    class_weights=class_weights,
    learning_rate=LR
)
model_wrapper.model.summary()

### Callbacks (EarlyStopping, ReduceLROnPlateau, ModelCheckpoint)

In [ ]:
callbacks = model_wrapper.setup_callbacks(model_name=str(EXP_DIR / 'chkpt'))

## 6 — Treinamento

In [ ]:
history = model_wrapper.model.fit(
    X_train_e, y_train,
    validation_data=(X_val_e, y_val),
    epochs=EPOCHS,
    batch_size=BATCH,
    shuffle=True,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1,
)

## 7 — Avaliação

In [ ]:
test_probs = model_wrapper.model.predict(X_test_e, batch_size=BATCH)
evaluator = ModelEvaluator(test_probs, y_test, threshold=-1, minPrecision=0.7)
evaluator.execute()
print(json.dumps(evaluator.metrics, indent=2))

## 8 — Salvando Artefatos

In [ ]:
model_wrapper.save_model(str(EXP_DIR / 'final_model'))
import pandas as pd
pd.DataFrame(history.history).to_csv(EXP_DIR / 'history.csv', index=False)

## 9 — Conclusão
Este notebook demonstra que a implementação em `codes/` atende a todos os requisitos da **Opção B** do *description.pdf*, incluindo:
* janela = 240 e formato de entrada `(window, 1)`,
* duas camadas `LSTM(64)` empilhadas com `BatchNormalization`, segue-se `Dense(64)` + BN e `Dense(1, sigmoid)`,
* callbacks `EarlyStopping`, `ReduceLROnPlateau` e `ModelCheckpoint`,
* perda `BinaryFocalCrossentropy` (ou BCE) com pesos de classe,
* métricas de Precisão, Recall, F1, e curvas ROC/PR.